In [ ]:
#Packages
import anndata

import tensorflow as tf
import numpy as np
from matplotlib import pyplot as plt
import h5py
import pandas as pd
import re
import os
import time
from sklearn.model_selection import train_test_split

#Import code
import encoder_decoder_builder_batch
import mask_builder
import model
import training_pVAE

In [ ]:
#Create Folder for ANN_data
anndata_loc='/cluster/work/grlab/projects/projects2020-autoencoder_pathway/gamma_VAE/kang_2018/kang_count.h5ad'
adata=anndata.read_h5ad(anndata_loc)

In [ ]:
geneSets=[]
for i in range(adata.varm["I"].shape[1]):
    geneSets.append(np.where(adata.varm["I"][:,i]==1)[0])

In [ ]:
auxillary_nodes=1
for i in range(auxillary_nodes):
    geneSets.append(np.arange(adata.X.shape[1]))

In [ ]:
#Train/test split
FPKM_train,FPKM_test=train_test_split(adata.X, shuffle=True,test_size=0.25)
FPKM_test.shape

In [ ]:
hidden_layers=[12]
mask_encoder,mask_decoder=mask_builder.mask_builder(input_dim=FPKM_test.shape[1],
                                                   hidden_layers=hidden_layers,
                                                   geneSets=geneSets
                                                   ,p_p_latent_dim=4)

decoder_1,decoder_2=encoder_decoder_builder_batch.decoder_builder(mask_decoder=mask_decoder,kernel_initializer="he_uniform"
                    ,bias_initializer="zero",activation_hidden="elu",bias_last_layer=False)

encoder=encoder_decoder_builder_batch.encoder_builder(mask_encoder=mask_encoder,kernel_initializer="he_uniform"
                    ,bias_initializer="zero",activation_hidden="elu")
vae_two_layer = model.VAE(encoder=encoder,decoder_1=decoder_1,decoder_2=decoder_2,dim=[FPKM_test.shape[1],len(geneSets)])
history,model_two = training_pVAE.train_VAE(training_data=FPKM_train,validation_data=FPKM_test,geneSets=geneSets,hidden=hidden_layers[0],
                                                                  batch_size=256,num_epochs=1200,
                                                                  model=vae_two_layer,beta=1e-05,
                                                                      optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
                                                                      verbose=True,auxillary_nodes=auxillary_nodes,full_loss=False)

In [ ]:
fig,ax1 = plt.subplots(1, dpi=200,constrained_layout=True)
plt.ylim()

ax1.plot(history["Loss"],label="Loss")
ax1.plot(history["Val Loss"],label="Validation Loss")
ax1.set(xlabel='Epoch',
       ylabel='Loss')

chartBox = ax1.get_position()
ax1.legend(loc='upper center', bbox_to_anchor=(1.3, 0.8), shadow=False, ncol=1,fontsize=8)


In [ ]:
x_reconstructed, mu, log_var,z,last_layer=model_two.call(adata.X)
z=np.asarray(z)
    
latent_sp=pd.DataFrame(z,columns=paths_ext)
latent_sp.index=adata.obs_names


In [ ]:
paths=list(adata.uns['terms'])
if 4>1:
    paths_ext=[]
    for i in range(len(paths)):
        for j in range(4):
            paths_ext.append(paths[i]+"_"+str(j))

    for i in range(4*1):
        paths_ext.append("auxiliary_"+str(i))
else:
    paths_ext=paths
    for i in range(4*1):
        paths_ext.append("auxiliary_"+str(i))

adata.obsm["Latent_sp"]=latent_sp
latent_sp=pd.DataFrame(z,columns=paths_ext)
latent_sp.index=adata.obs_names
adata.obsm["Latent_sp"]=latent_sp

In [ ]:
adata.write("/cluster/work/grlab/projects/projects2020-autoencoder_pathway/gamma_VAE/kang_2018/Kang_test_run.h5ad")